In [1]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils
from sklearn.model_selection import train_test_split


from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

questions = pd.read_csv("data/train_dataset.csv", header=None, encoding="iso-8859-1", sep=";")
questions.columns = ['id', 'question', 'answer', 'topic']



REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

questions['question'] = questions['question'].apply(clean_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/silvan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


In [14]:
X = questions.question
y = questions.topic
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

train_posts = X_train
train_tags = y_train

test_posts = X_test
test_tags = y_test

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

tokenize.fit_on_texts(train_posts) # only fit on train
x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)


encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

batch_size = 32
epochs = 2


# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)


val_score = model.evaluate(x_train, y_train,
                       batch_size=batch_size, verbose=1)
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Train acc:',val_score[1] )
print('Test accuracy:', score[1])

Train on 6417 samples, validate on 713 samples
Epoch 1/2
6417/6417 [==============================] - 2s 300us/sample - loss: 0.7976 - categorical_accuracy: 0.7290 - val_loss: 0.4151 - val_categorical_accuracy: 0.8752
Epoch 2/2
1783/1783 [==============================] - 0s 74us/sample - loss: 0.3702 - categorical_accuracy: 0.8833
Train acc: 0.9354839
Test accuracy: 0.8833427


In [16]:
generated_questions = pd.read_csv("data/crowdanswers.tsv", encoding="ISO-8859-1", delimiter="\t", na_filter=False)
generated_questions.columns = ['id', 'question', 'answer', 'difficulty', 'opinion', 'factuality']

labels = sorted(['science-technology', 'for-kids', 'video-games', 'sports', 'music'])

generated_questions['question'].apply(clean_text)
x_predict = tokenize.texts_to_matrix(generated_questions['question'])
# result = model.predict(x_predict)
result= model.predict_classes(x_predict, batch_size=1)
#y_classes = result.argmax(axis=-1)
predictated_labels = [labels[i] for i in result]
# predicted_label = sorted(labels)[result]
output = pd.DataFrame(data={"id":generated_questions["id"], "question":generated_questions["question"], "answer":generated_questions["answer"], "difficulty":generated_questions["difficulty"], "opinion":generated_questions["opinion"], "factuality":generated_questions["factuality"], "topic":predictated_labels})
output.to_csv('generated.csv',encoding='utf-8', index=False, header=True)



In [51]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
from gensim.models import KeyedVectors
from keras.backend.tensorflow_backend import set_session
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.layers import Dense, Embedding, Dropout, Flatten, Activation
from tensorflow.python.keras.layers import LSTM, SimpleRNN
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.preprocessing import text as keras_text, sequence as keras_seq
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score
from sklearn.utils import shuffle
from tensorflow import set_random_seed
import gc
import os
from keras.preprocessing.sequence import pad_sequences

EMBEDDING_SIZE=300
WORDS_SIZE=8000
INPUT_SIZE=100
NUM_CLASSES=5
EPOCHS=10

questions = pd.read_csv("data/train_dataset.csv", header=None, encoding="iso-8859-1", sep=";")
questions.columns = ['id', 'question', 'answer', 'topic']

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

questions['question'] = questions['question'].apply(clean_text)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)


tokenizer = keras_text.Tokenizer(num_words=2000,char_level=False)
tokenizer.fit_on_texts(x_train)
#tokenizer.num_words=WORDS_SIZE

maxlen = 100
## Tokkenizing train data and create matrix
list_tokenized_train = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(list_tokenized_train, padding='post', maxlen=maxlen)

## Tokkenizing test data and create matrix
list_tokenized_test = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(list_tokenized_test, padding='post', maxlen=maxlen)

encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

y_train = utils.to_categorical(y_train, NUM_CLASSES)
y_test = utils.to_categorical(y_test, NUM_CLASSES)

In [41]:
word2vec = KeyedVectors.load_word2vec_format('word2vec/GoogleNews-vectors-negative300.bin.gz', binary=True)
word_index = tokenizer.word_index

In [48]:
googlenews_w2v_matrix = np.zeros((len(word_index) + 1, 300))

for word,i in word_index.items():
  if word in word2vec.vocab:
    googlenews_w2v_matrix[i] = word2vec[word]

print(len(googlenews_w2v_matrix))
print(len(word_index)+1)
embedding_layer = Embedding(len(word_index)+1,
                            300,
                            weights=[googlenews_w2v_matrix],
                            input_length=INPUT_SIZE)

20298
20298


In [53]:
model = Sequential(name='Word2Vec LSTM')

model.add(embedding_layer)
model.add(LSTM(70))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [54]:
history = model.fit(x = x_train,
          y = y_train,
          validation_data = (x_test, y_test),
          epochs = EPOCHS,
          batch_size = 128,
          verbose =1,shuffle=True)


val_score = model.evaluate(x_train, y_train, batch_size=batch_size, verbose=1,)
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

Train on 7130 samples, validate on 1783 samples
Epoch 1/10
7130/7130 [==============================] - 25s 4ms/sample - loss: 1.4177 - categorical_accuracy: 0.4003 - val_loss: 1.3590 - val_categorical_accuracy: 0.4173
Epoch 2/10
7130/7130 [==============================] - 24s 3ms/sample - loss: 1.3838 - categorical_accuracy: 0.4111 - val_loss: 1.3578 - val_categorical_accuracy: 0.4173
Epoch 3/10
7130/7130 [==============================] - 24s 3ms/sample - loss: 1.3838 - categorical_accuracy: 0.4111 - val_loss: 1.3616 - val_categorical_accuracy: 0.4173
Epoch 4/10
7130/7130 [==============================] - 24s 3ms/sample - loss: 1.3810 - categorical_accuracy: 0.4111 - val_loss: 1.3584 - val_categorical_accuracy: 0.4173
Epoch 5/10
7130/7130 [==============================] - 24s 3ms/sample - loss: 1.3814 - categorical_accuracy: 0.4111 - val_loss: 1.3550 - val_categorical_accuracy: 0.4173
Epoch 6/10
7130/7130 [==============================] - 24s 3ms/sample - loss: 1.3829 - categoric

KeyboardInterrupt: 

In [ ]:
generated_questions = pd.read_csv("data/crowdanswers.tsv", encoding="ISO-8859-1", delimiter="\t", na_filter=False)
generated_questions.columns = ['id', 'question', 'answer', 'difficulty', 'opinion', 'factuality']

labels = sorted(['science-technology', 'for-kids', 'video-games', 'sports', 'music'])

list_tokenized = tokenizer.texts_to_sequences(generated_questions['question'])
x_predict = pad_sequences(list_tokenized, padding='post', maxlen=maxlen)

result= model.predict_classes(x_predict, batch_size=128)
predictated_labels = [labels[i] for i in result]
print(predictated_labels)
# predicted_label = sorted(labels)[result]
output = pd.DataFrame(data={"id":generated_questions["id"], "question":generated_questions["question"], "answer":generated_questions["answer"], "difficulty":generated_questions["difficulty"], "opinion":generated_questions["opinion"], "factuality":generated_questions["factuality"], "topic":predictated_labels})
output.to_csv('data/classified.csv',encoding='utf-8', index=False)

In [26]:
questions = pd.read_csv("data/classified.csv",
                          encoding="utf-8", sep=",", error_bad_lines=False)
questions = questions.groupby('question').filter(
    lambda x: x['factuality'].sum() < 1).reset_index()

from collections import Counter

def majority(lst):
    data = Counter(lst)
    return max(lst, key=data.get)


questions = questions.groupby(['question'])['difficulty'].agg(majority)

print(questions)

question
 'El ClÃÂ¡sico' is a game between which club giants?                                                                                                   Easy
 How many players, including the goaltender, make up an ice hockey team?                                                                                Hard
7 rings' is a song by which American singer?                                                                                                          Medium
A device used to measure the strength of magnetic field.                                                                                                Hard
A winner in Boxing (Light Flyweight) at the 2012 Summer Olympics? He won a gold medal in the 2008 Summer Olympics as well.                              Hard
According to esportsearnings.com 2019 what is the top prize money awarding game of all time? - "League of Legends"? "World of Warcraft"? "Dota 2"?    Medium
According to the ABRSM top list, what is the most